In [5]:
import json

# Load the JSON data from file
with open('OpenVLM.json', 'r') as file:
    data = json.load(file)

# Extract the required information
results = data['results']

# Prepare data for export
export_data = []

for model_name, model_data in results.items():
    meta = model_data['META']
    model_info = {
        "Model": model_name,
        "Source": meta['Method'][1],
        "Parameters": meta['Parameters'],
        "Time": meta['Time'],
        "OpenSource": meta['OpenSource']
    }
    
    # Collect Overall scores for each test
    for test_name, test_data in model_data.items():
        if test_name != 'META' and 'Overall' in test_data:
            overall_score = test_data['Overall']
            if isinstance(overall_score, str):
                # Handle cases like "62.3 (PPL)"
                overall_score = float(overall_score.split()[0])
            model_info[f"{test_name} Overall"] = overall_score
    
    export_data.append(model_info)

# Export to JSON
with open('processed_model_data.json', 'w') as f:
    json.dump(export_data, f, indent=2)

print("Data exported to 'processed_model_data.json'")

Data exported to 'processed_model_data.json'


In [3]:
#count number of models
print("Number of models: ", len(export_data))

Number of models:  92


In [10]:
import json
import csv

# Load the JSON data from file
with open('processed_model_data.json', 'r') as file:
    data = json.load(file)

# Define the fields we want to keep and their new names
fields_to_keep = {
    'Model': 'Model',
    'Parameters': 'ModelSize',
    'OpenSource': 'OpenSource',
    'TextVQA_VAL Overall': 'TextVQA',
    'POPE Overall': 'POPE',
    'AI2D Overall': 'AI2D',
    'ChartQA_TEST Overall': 'ChartQA',
    'ScienceQA_TEST Overall': 'ScienceQA',
    'MMMU_VAL Overall': 'MMMU',
    'MMBench_TEST_EN Overall': 'MMBench',
    'MathVista Overall': 'MathVista'
}

# Prepare reduced data
reduced_data = []

for item in data:
    reduced_item = {}
    scores = []
    for old_key, new_key in fields_to_keep.items():
        if old_key in item:
            value = item[old_key]
            reduced_item[new_key] = value
            if new_key not in ['Model', 'ModelSize', 'OpenSource'] and value != 'N/A':
                try:
                    scores.append(float(value))
                except ValueError:
                    pass  # Ignore if conversion to float fails
        else:
            reduced_item[new_key] = 'N/A'
    
    # Calculate average score
    if scores:
        reduced_item['AverageScore'] = sum(scores) / len(scores)
    else:
        reduced_item['AverageScore'] = 'N/A'
    
    reduced_data.append(reduced_item)

# Export to JSON
with open('reduced_model_data.json', 'w') as f:
    json.dump(reduced_data, f, indent=2)

print("Reduced data exported to 'reduced_model_data.json'")

# Export to CSV
with open('reduced_model_data.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=list(fields_to_keep.values()) + ['AverageScore'])
    writer.writeheader()
    for item in reduced_data:
        writer.writerow(item)

print("Reduced data exported to 'reduced_model_data.csv'")

Reduced data exported to 'reduced_model_data.json'
Reduced data exported to 'reduced_model_data.csv'
